In [30]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import read_player_stats
import training_data
from sklearn import linear_model
from sklearn import cross_validation
from sklearn import grid_search
from sklearn import metrics
from sklearn.learning_curve import learning_curve
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 250

[autoreload of training_data failed: Traceback (most recent call last):
  File "/usr/local/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
  File "training_data.py", line 55
    y_proj model.predict(X_proj)
               ^
SyntaxError: invalid syntax
]
[autoreload of training_data failed: Traceback (most recent call last):
  File "/usr/local/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
  File "training_data.py", line 55
    y_proj model.predict(X_proj)
               ^
SyntaxError: invalid syntax
]
[autoreload of training_data failed: Traceback (most recent call last):
  File "/usr/local/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
  File "training_data.py", line 55
    y_proj model.predict(X_proj)
               ^
SyntaxError: invalid syntax
]
[

In [5]:
#grab total data for all positions
qb_total_df = training_data.make_total_data(seasons=range(2004,2015), pages=[0,1], pos='qb')
rb_total_df = training_data.make_total_data(seasons=range(2004,2015), pages=[0,1,2,3], pos='rb')
wr_total_df = training_data.make_total_data(seasons=range(2004,2015), pages=[0,1,2,3], pos='wr')
te_total_df = training_data.make_total_data(seasons=range(2004,2015), pages=[0,1], pos='te')

In [6]:
#training sets
qb_train_df = training_data.make_training_df(qb_total_df, seasons=range(2004,2014), ppg=True)
rb_train_df = training_data.make_training_df(rb_total_df, seasons=range(2004,2014), ppg=True)
wr_train_df = training_data.make_training_df(wr_total_df, seasons=range(2004,2014), ppg=True)
te_train_df = training_data.make_training_df(te_total_df, seasons=range(2004,2014), ppg=True)

In [7]:
qb_total_df.head()

,Name,Team,Games,PassComp,PassAtt,PassYards,PassTD,INT,RunAtt,RunYards,RunTD,FFP,FFPPG,Season
28,AJFeeley,MIA,11,191,356,1893,11,15,14,13,1,146.0,13.3,2004
164,AJFeeley,MIA,2,0,0,0,0,0,0,0,0,0.0,0.0,2005
224,AJFeeley,PHI,2,26,38,342,3,0,1,3,0,29.4,14.7,2006
304,AJFeeley,PHI,4,59,103,681,5,8,7,23,0,56.4,14.1,2007
637,AJFeeley,STL,5,53,97,548,1,2,3,4,0,31.8,6.4,2011


In [8]:
#train one regression model
X_train = np.array(qb_train_df.drop(['Name','FFPPG'], axis=1))
y_train = np.array(qb_train_df['FFPPG'])

parameters = {'alpha': np.logspace(-5,5,num=30)}
lin_model = grid_search.GridSearchCV(linear_model.Ridge(normalize=True), parameters, cv=10)
lin_model.fit(X_train, y_train)

scores =  cross_validation.cross_val_score(lin_model.best_estimator_, X_train, y_train, cv=10, scoring='mean_absolute_error')
scores

array([-5.19642518, -4.96827708, -4.85902221, -5.32455082, -4.78903028,
       -3.57224926, -4.76286332, -4.39907412, -4.39593469, -4.60734357])

In [14]:
lin_model.best_estimator_

Ridge(alpha=0.30391953823132012, copy_X=True, fit_intercept=True,
   max_iter=None, normalize=True, solver='auto', tol=0.001)

In [22]:
qb_model = training_data.train_player_model(qb_train_df)
rb_model = training_data.train_player_model(rb_train_df)
wr_model = training_data.train_player_model(wr_train_df)
te_model = training_data.train_player_model(te_train_df)

In [28]:
#projections for 2015
qb_most_rec = training_data.data_for_projection(qb_total_df, season=2015)

In [29]:
qb_proj = training_data.ff qb_most_rec

,Name,Games_1,PassComp_1,PassAtt_1,PassYards_1,PassTD_1,INT_1,RunAtt_1,RunYards_1,RunTD_1,FFP_1,FFPPG_1,Games_2,PassComp_2,PassAtt_2,PassYards_2,PassTD_2,INT_2,RunAtt_2,RunYards_2,RunTD_2,FFP_2,FFPPG_2
0,AaronRodgers,9,193,290,2536,17,6,30,120,0,206.8,23.0,16,341,520,4381,38,5,43,269,2,410.0,25.6
1,AlexSmith,15,308,509,3313,23,8,75,432,1,306.9,20.5,15,302,463,3268,18,6,50,254,1,266.8,17.8
2,AndrewLuck,16,345,572,3830,23,9,63,377,4,345.2,21.6,16,380,616,4761,40,16,64,263,3,442.4,27.6
3,AndyDalton,16,363,586,4296,33,20,61,183,2,377.1,23.6,16,310,483,3398,19,17,60,169,4,286.8,17.9
4,BenRoethlisberger,16,375,584,4261,28,14,26,99,1,341.0,21.3,16,408,608,4952,32,9,32,27,0,378.3,23.6
5,BlaineGabbert,3,42,86,481,1,7,9,32,0,31.3,10.4,1,3,7,38,1,0,1,5,0,6.4,6.4
6,BrandonWeeden,8,141,267,1731,9,9,12,44,0,127.0,15.9,4,24,41,303,3,2,6,-6,0,26.6,6.6
7,BrianHoyer,3,57,96,615,5,3,6,16,0,52.4,17.5,14,244,440,3339,12,13,25,32,0,218.2,15.6
8,BrockOsweiler,2,11,16,95,0,0,3,2,0,5.0,2.5,4,4,10,52,1,0,8,0,0,6.6,1.7
9,CamNewton,16,292,473,3379,24,13,111,587,6,359.7,22.5,14,262,448,3127,18,12,103,539,5,312.3,22.3
